Model-based VI (Vibration analysis) is a technique used to identify the type and severity of a fault in a machine by analyzing its vibration signals. It involves building a model of the machine's vibration signature under normal operating conditions and comparing it with the actual vibration signals during operation to detect any deviations.

Dataset :
each file has 10000 3-phase current readings. 
There are 317 files, which means the readings were taken for 317 seconds. 
There are 3 columns in each file represent the 3 currents.

In [24]:
import numpy as np
from scipy.fftpack import fft, fftfreq, rfft, rfftfreq
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import copy

In [23]:
data = pd.read_csv('combined_data.csv')


To perform model-based VI on the given dataset, we can follow these steps:

1.Load the dataset into a pandas DataFrame.

2.Compute the FFT of each column (corresponding to each phase) to convert the time-domain signal to the frequency domain.

3.Use a sliding window to segment the signal into smaller windows of length L.

4.For each window, compute the FFT and calculate the magnitude spectrum.

5.Use the magnitude spectrum to estimate the model parameters using a curve-fitting algorithm. This will give us a model of the machine's 
vibration signature under normal operating conditions.

6.During machine operation, repeat steps 2-4 for each window of data and compare the current vibration signature to the model created in step 5.

7.If the current vibration signature deviates significantly from the model, it indicates the presence of a fault.

In [27]:
# Load the data into a pandas DataFrame
df = copy.deepcopy(data)

In [30]:
# Define the length of the sliding window
L = 1000

# Define the curve-fitting function
def model_func(freq, amp, freq0, sigma):
    return amp * np.exp(-0.5 * ((freq - freq0) / sigma) ** 2)


In [31]:

# Initialize lists to store the model parameters for each phase
phase_params = {'phase_A': {'amp': [], 'freq0': [], 'sigma': []},
                'phase_B': {'amp': [], 'freq0': [], 'sigma': []},
                'phase_C': {'amp': [], 'freq0': [], 'sigma': []}}


The curve fitting function used in the example code is a Gaussian function defined by the parameters amp, freq0, and sigma, which represent the amplitude, central frequency, and width of the peak, respectively. The function was chosen based on its ability to fit the magnitude spectrum of the vibration signal, which typically exhibits a dominant frequency peak.

The Gaussian function is a commonly used curve in modeling vibration signals because it can accurately capture the shape of the frequency spectrum near a dominant frequency peak. Additionally, the curve fitting function can be customized based on the characteristics of the signal being analyzed.

In the code, the curve_fit function from the scipy.optimize module is used to fit the Gaussian function to the magnitude spectrum of each window of data. The popt parameter represents the optimal values for the model parameters amp, freq0, and sigma, which are then stored for later analysis.

Here same function was fit for all 3 currents


In [73]:

# Loop over the windows of data
for i in range(0, len(df), L):
    # Extract a window of data for each phase
    window_A = df.loc[i:i+L-1, 'current_1']
    window_B = df.loc[i:i+L-1, 'current_2']
    window_C = df.loc[i:i+L-1, 'current_3']
    
    # Compute the FFT of each phase
    fft_A = np.fft.fft(window_A)
    fft_B = np.fft.fft(window_B)
    fft_C = np.fft.fft(window_C)
    
    # Compute the magnitude spectrum of each phase
    mag_spectrum_A = np.abs(fft_A)
    mag_spectrum_B = np.abs(fft_B)
    mag_spectrum_C = np.abs(fft_C)
    
    # Compute the frequency axis
    freq_axis = np.fft.fftfreq(L, 1/len(df))
    
    # Fit the model to the spectrum of each phase
    for phase, mag_spectrum in zip(['phase_A', 'phase_B', 'phase_C'], 
                                   [mag_spectrum_A, mag_spectrum_B, mag_spectrum_C]):
        popt, pcov = curve_fit(model_func, freq_axis, mag_spectrum.ravel(),maxfev=10000)
        
        # Store the model parameters for each phase
        phase_params[phase]['amp'].append(popt[0])
        phase_params[phase]['freq0'].append(popt[1])
        phase_params[phase]['sigma'].append(popt[2])

# Calculate the mean and standard deviation of the model parameters for each phase
for phase in ['phase_A', 'phase_B', 'phase_C']:
    mean_amp = np.mean(phase_params[phase]['amp'])
    std_amp = np.std(phase_params[phase]['amp'])
    mean_freq0 = np.mean(phase_params[phase]['freq0'])
    std_freq0 = np.std(phase_params[phase]['freq0'])
    mean_sigma = np.mean(phase_params[phase]['sigma'])
    std_sigma = np.std(phase_params[phase]['sigma'])
    print(f"Model parameters for {phase}:")
    print(f"Amplitude: {mean_amp} +/- {std_amp}")
    print(f"Frequency: {mean_freq0} +/- {std_freq0}")
    print(f"Bandwidth: {mean_sigma} +/- {std_sigma}")


Model parameters for phase_A:
Amplitude: 1982430.6281658122 +/- 11860.058750644035
Frequency: -3.5153052876403025e-05 +/- 0.0012293073988161307
Bandwidth: 933.9719383455442 +/- 68.77748860261754
Model parameters for phase_B:
Amplitude: 2167703.873363091 +/- 11192.692075283932
Frequency: -0.03576042510173978 +/- 2.2250749703774724
Bandwidth: 928.266857762362 +/- 50.31088491947947
Model parameters for phase_C:
Amplitude: 430504.24940510694 +/- 38387.03631331259
Frequency: 0.0029345399561199388 +/- 0.11031701155672653
Bandwidth: 1374.0974583676143 +/- 2403.076112570671


3 different functions for 3 currents

In [88]:
# Define the length of the sliding window
L = 1000

# Define the curve-fitting functions for each current
def model_func_I1(freq, amp, freq0, sigma):
    return amp * np.exp(-0.5 * ((freq - freq0) / sigma) ** 2)

def model_func_I2(freq, amp, freq0, sigma):
    return amp * np.exp(-0.5 * ((freq - freq0) / sigma) ** 2)

def model_func_I3(freq, amp, freq0, sigma):
    return amp * np.exp(-0.5 * ((freq - freq0) / sigma) ** 2)

# Initialize lists to store the model parameters for each current
amp_list_I1 = []
freq0_list_I1 = []
sigma_list_I1 = []

amp_list_I2 = []
freq0_list_I2 = []
sigma_list_I2 = []

amp_list_I3 = []
freq0_list_I3 = []
sigma_list_I3 = []

# Loop over the windows of data
for i in range(0, len(df), L):
    # Extract a window of data for each current
    I1_window = df.loc[i:i+L-1, 'current_1']
    I2_window = df.loc[i:i+L-1, 'current_2']
    I3_window = df.loc[i:i+L-1, 'current_3']
    
    # Compute the FFT of each current
    I1_fft_data = np.fft.fft(I1_window)
    I2_fft_data = np.fft.fft(I2_window)
    I3_fft_data = np.fft.fft(I3_window)
    
    # Compute the magnitude spectrum for each current
    I1_mag_spectrum = np.abs(I1_fft_data)
    I2_mag_spectrum = np.abs(I2_fft_data)
    I3_mag_spectrum = np.abs(I3_fft_data)
    
    # Compute the frequency axis for each current
    freq_axis = np.fft.fftfreq(L, 1/len(df))
    
    # Fit the model to the spectrum for each current
    popt_I1, pcov_I1 = curve_fit(model_func_I1, freq_axis, I1_mag_spectrum.ravel(),maxfev=10000)
    popt_I2, pcov_I2 = curve_fit(model_func_I2, freq_axis, I2_mag_spectrum.ravel(),maxfev=10000)
    popt_I3, pcov_I3 = curve_fit(model_func_I3, freq_axis, I3_mag_spectrum.ravel(),maxfev=10000)
    
    # Store the model parameters for each current
    amp_list_I1.append(popt_I1[0])
    freq0_list_I1.append(popt_I1[1])
    sigma_list_I1.append(popt_I1[2])
    
    amp_list_I2.append(popt_I2[0])
    freq0_list_I2.append(popt_I2[1])
    sigma_list_I2.append(popt_I2[2])
    
    amp_list_I3.append(popt_I3[0])
    freq0_list_I3.append(popt_I3[1])
    sigma_list_I3.append(popt_I3[2])

# Calculate the mean and standard deviation of the model parameters for each current
mean_amp_I1 = np.mean(amp_list_I1)
std_amp_I1 = np.std(amp_list_I1)
mean_freq0_I1 = np.mean(freq0_list_I1)
std_freq0_I1 = np.std(freq0_list_I1)

mean_amp_I2 = np.mean(amp_list_I2)
std_amp_I2 = np.std(amp_list_I2)
mean_freq0_I2 = np.mean(freq0_list_I2)
std_freq0_I2 = np.std(freq0_list_I2)

mean_amp_I3 = np.mean(amp_list_I3)
std_amp_I3 = np.std(amp_list_I3)
mean_freq0_I3 = np.mean(freq0_list_I3)
std_freq0_I3 = np.std(freq0_list_I3)

In [84]:
an = pd.read_csv('../anomalous_data/anomalous_data.csv')

In [85]:

# Loop over the windows of data
for i in range(0, len(an), L):
    # Extract a window of data for each phase
    window_A = an.loc[i:i+L-1, 'current_1']
    window_B = an.loc[i:i+L-1, 'current_2']
    window_C = an.loc[i:i+L-1, 'current_3']
    
    # Compute the FFT of each phase
    fft_A = np.fft.fft(window_A)
    fft_B = np.fft.fft(window_B)
    fft_C = np.fft.fft(window_C)
    
    # Compute the magnitude spectrum of each phase
    mag_spectrum_A = np.abs(fft_A)
    mag_spectrum_B = np.abs(fft_B)
    mag_spectrum_C = np.abs(fft_C)
    
    # Compute the frequency axis
    freq_axis = np.fft.fftfreq(L, 1/len(df))
    
    # Fit the model to the spectrum of each phase
    for phase, mag_spectrum in zip(['phase_A', 'phase_B', 'phase_C'], 
                                   [mag_spectrum_A, mag_spectrum_B, mag_spectrum_C]):
        popt, pcov = curve_fit(model_func, freq_axis, mag_spectrum.ravel(),maxfev=10000)
        
        # Store the model parameters for each phase
        phase_params[phase]['amp'].append(popt[0])
        phase_params[phase]['freq0'].append(popt[1])
        phase_params[phase]['sigma'].append(popt[2])

# Calculate the mean and standard deviation of the model parameters for each phase
for phase in ['phase_A', 'phase_B', 'phase_C']:
    mean_amp = np.mean(phase_params[phase]['amp'])
    std_amp = np.std(phase_params[phase]['amp'])
    mean_freq0 = np.mean(phase_params[phase]['freq0'])
    std_freq0 = np.std(phase_params[phase]['freq0'])
    mean_sigma = np.mean(phase_params[phase]['sigma'])
    std_sigma = np.std(phase_params[phase]['sigma'])
    print(f"Model parameters for {phase}:")
    print(f"Amplitude: {mean_amp} +/- {std_amp}")
    print(f"Frequency: {mean_freq0} +/- {std_freq0}")
    print(f"Bandwidth: {mean_sigma} +/- {std_sigma}")


Model parameters for phase_A:
Amplitude: 1981919.851588493 +/- 33927.306965395204
Frequency: -0.000430841329611471 +/- 0.02465569642107601
Bandwidth: 933.8169353722102 +/- 69.44187702403082
Model parameters for phase_B:
Amplitude: 2167145.720890607 +/- 36494.053038011385
Frequency: -0.03459132063262363 +/- 2.225977148136065
Bandwidth: 928.138818219723 +/- 50.931583598065345
Model parameters for phase_C:
Amplitude: 430393.34990941035 +/- 38997.6543134709
Frequency: 0.006905938214835358 +/- 0.2706504639130529
Bandwidth: 1373.7940338210578 +/- 2402.8401376373017


c:\Users\deepdesk\anaconda3\lib\site-packages\scipy\optimize\minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [ ]:
# Loop over the windows of data to detect faults
for i in range(0, len(df)-L, L):
    # Extract a window of data
    window = fft_data[i:i+L,:]
    
    # Compute the magnitude spectrum
    mag_spectrum = np.abs(window)
    
    # Compute the frequency axis
    freq_axis = np.fft.fftfreq(L, 1/len(df))
    
    # Fit the model to the spectrum
    popt, pcov = curve_fit(model_func, freq_axis, mag_spectrum.ravel())
    
    # Check for deviation from the model
    if (abs(popt[0]-mean_amp) > 2*std_amp or
        abs(popt[1]-mean_freq0) > 2*std_freq0 or
        abs(popt[2]-mean_sigma) > 2*std_sigma):
        print(f"Fault detected at time {i} seconds")

generic method

In [ ]:
import numpy as np
from vibtools import models, signal
from sklearn.preprocessing import StandardScaler

# Load the data
data = np.load('current_data.npy')

# Extract the vibration signals using MCSA
vibration_data = _get_vibration_data(data)

# Preprocess the vibration signals
filtered_data = signal.butterworth(vibration_data, fs=20000, fc=1000, btype='lowpass')
resampled_data = signal.resample(filtered_data, n_samples=10000)
denoised_data = signal.denoise(resampled_data, wavelet='sym8', level=5)

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(denoised_data)

# Build a model of the vibration signature under normal operating conditions
model = models.KMeansModel(scaled_data, n_clusters=1)

# Analyze the vibration signals during operation using the model
deviation = model.score_samples(scaled_data)

# Identify the type and severity of the fault based on the deviations detected
if deviation > threshold:
    print('Fault detected: Type and severity of fault')
else:
    print('No fault detected')
